In [ ]:
import geopandas as gpd

In [ ]:
DIR_DATA='https://raw.githubusercontent.com/meade68/DACSS_603D_Week04_HW1/refs/heads/main/DATA/'
JSON_WORLDINDICATORS = 'worldindicators.json'

In [ ]:
#read world
WORLD=gpd.read_file(DIR_DATA+JSON_WORLDINDICATORS)

#filter to a specific region
EUROPE= WORLD[ WORLD['region']== 'EUROPE' ]
EUROPE.plot()
EUROPE.head()

In [ ]:
LYR_DOT_DENSITY= EUROPE.copy()
#LYR_DOT_DENSITY.head()
LYR_DOT_DENSITY.set_index('Country',inplace=True)
LYR_DOT_DENSITY['Dot_density']= (EUROPE['mobiles'] / 25000).astype(int).values 
LYR_DOT_DENSITY['geometry']= LYR_DOT_DENSITY.sample_points(size=LYR_DOT_DENSITY['Dot_density'],rng=123)

In [ ]:
 EUROPE=EUROPE.to_crs(3035)

In [ ]:
BASE= EUROPE.to_crs(3035)
LYR_DOT_DENSITY = LYR_DOT_DENSITY.to_crs(3035)

BASE_MAP=BASE.plot(facecolor='white',edgecolor='lightblue')
LYR_DOT_DENSITY.to_crs(3035).plot(markersize=.02,color='darkblue',ax=BASE_MAP,alpha=.2)


In [ ]:




LYR_DOT_DENSITY.to_file('EUROPE_3035.gpkg',driver='GPKG',layer='DOT_DENSITY')
BASE.to_file('EUROPE_3035.gpkg',driver='GPKG',layer='BASEMAP')

In [ ]:
LYR_CHORO= EUROPE.copy()
LYR_CHORO['mobiles_pcapita']= LYR_CHORO.mobiles/LYR_CHORO.population

In [ ]:
#Begin Choropleth stuff.

import mapclassify
import numpy as np
import pandas as pd

np.random.seed(123)

#five intervals
K=5
classify=LYR_CHORO['mobiles_pcapita']

ei5=mapclassify.EqualInterval(classify)
q5=mapclassify.Quantiles(classify,k=K)
msd = mapclassify.StdMean(classify)

# based on similarity, good for multimodal data
mb5 = mapclassify.MaximumBreaks(classify, k=K)
# based on similarity, good for skewed data
ht = mapclassify.HeadTailBreaks(classify) # no K needed
# based on similarity, optimizer
fj5 = mapclassify.FisherJenks(classify, k=K)
# based on similarity, optimizer
jc5 = mapclassify.JenksCaspall(classify, k=K)
# based on similarity, optimizer
mp5 = mapclassify.MaxP(classify, k=K)



class5 = ei5,msd, q5,mb5,  ht, fj5, jc5, mp5
# Collect ADCM for each classifier
fits = np.array([ c.adcm for c in class5])
# Convert ADCM scores to a DataFrame
adcms = pd.DataFrame(fits)
# Add classifier names
adcms['classifier'] = [c.name for c in class5]
# Add column names to the ADCM
adcms.columns = ['ADCM', 'Classifier']


adcms.sort_values('ADCM').plot.barh(x='Classifier')

In [ ]:
#keep the lowest
LYR_CHORO['mobiles_pcapita_FJ5']=fj5.yb
LYR_CHORO

In [ ]:
# renaming
newLabelsForLevels={0:"0_VeryLow", 1:"1_Low", 2:"2_Middle", 3:"3_High", 4:"4_VeryHigh"}

LYR_CHORO['mobiles_pcapita_FJ5_cat']=LYR_CHORO.loc[:,'mobiles_pcapita_FJ5'].replace(newLabelsForLevels)

# we have
LYR_CHORO.head(20)

In [ ]:
#Save to file
LYR_CHORO.to_file('EUROPE_3035.gpkg',driver='GPKG',layer='CHOROPLETH')

In [ ]:
##Startt proportional symbol stuff
import numpy as np
LYR_PROP_SYMBOL= EUROPE.copy()
max_size=40
min_size=1

scaled_sizes = np.sqrt(
    (LYR_PROP_SYMBOL.mobiles - LYR_PROP_SYMBOL.mobiles.min()) / (LYR_PROP_SYMBOL.mobiles.max() - LYR_PROP_SYMBOL.mobiles.min())
) * (max_size-1) + min_size

scaled_sizes.sort_values()
LYR_PROP_SYMBOL['size']= scaled_sizes.values

In [ ]:
LYR_PROP_SYMBOL.set_index('Country',inplace=True)

#this is just a copy (new index names)
LYR_PROP_SYMBOL.head()

In [ ]:

LYR_PROP_SYMBOL['geometry']=LYR_PROP_SYMBOL['geometry'].centroid




In [ ]:
LYR_PROP_SYMBOL

In [ ]:
LYR_PROP_SYMBOL.plot(
    markersize=LYR_PROP_SYMBOL['size'],
    color='black'
)

In [ ]:
LYR_PROP_SYMBOL.to_file('EUROPE_3035.gpkg',driver='GPKG',layer='PROP_SYMBOL')